## data_preparation.ipynb

### ==============================
### Phase 3: Data Preparation
### ==============================



In [1]:
# --- 1. Import Libraries ---
import pandas as pd
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# --- 2. Load Data ---

events = pd.read_csv('/content/drive/MyDrive/data/TMP/events.csv')
category_tree = pd.read_csv('/content/drive/MyDrive/data/TMP/category_tree.csv')
df_prop1  = pd.read_csv('/content/drive/MyDrive/data/TMP/item_properties_part1.1.csv')
df_prop2 = pd.read_csv('/content/drive/MyDrive/data/TMP/item_properties_part2.csv')

item_props = pd.concat([df_prop1, df_prop2])

In [4]:
# --- 3. Convert timestamps to datetime ---
def to_datetime(ts):
    return datetime.utcfromtimestamp(int(ts)/1000)

events['datetime'] = events['timestamp'].apply(to_datetime)
item_props['datetime'] = item_props['timestamp'].apply(to_datetime)

In [5]:
# --- 4. Basic Cleaning ---
# Drop duplicates
events.drop_duplicates(inplace=True)
item_props.drop_duplicates(inplace=True)

# Remove any events with missing IDs
events = events.dropna(subset=['visitorid', 'itemid'])
item_props = item_props.dropna(subset=['itemid', 'property', 'value'])

In [6]:
# --- 5. Handle Item Properties ---
# Since properties change over time, keep the latest property snapshot before each event
# Sort both datasets
events = events.sort_values(by=['visitorid', 'datetime'])
item_props = item_props.sort_values(by=['itemid', 'datetime'])

# Optional: Filter only important properties (like categoryid, available)
important_props = item_props[item_props['property'].isin(['categoryid', 'available'])]

# Pivot properties to wide format for merging
item_props_pivot = important_props.pivot_table(
    index=['itemid'],
    columns='property',
    values='value',
    aggfunc='last'  # Take the latest value
).reset_index()

In [7]:
# --- 5. Handle Item Properties ---
# Since properties change over time, keep the latest property snapshot before each event
# Sort both datasets
events = events.sort_values(by=['visitorid', 'datetime'])
item_props = item_props.sort_values(by=['itemid', 'datetime'])

# Optional: Filter only important properties (like categoryid, available)
important_props = item_props[item_props['property'].isin(['categoryid', 'available'])]

# Pivot properties to wide format for merging
item_props_pivot = important_props.pivot_table(
    index=['itemid'],
    columns='property',
    values='value',
    aggfunc='last'  # Take the latest value
).reset_index()

In [8]:
# --- 6. Merge Events with Item Properties ---
events_merged = pd.merge(events, item_props_pivot, on='itemid', how='left')

In [9]:
# --- 7. Add Category Hierarchy ---
# Map categoryid to its parent from category_tree
category_tree.columns = ['categoryid', 'parentid']
# Convert categoryid in category_tree to string to match events_merged
category_tree['categoryid'] = category_tree['categoryid'].astype(str)
events_merged = pd.merge(events_merged, category_tree, on='categoryid', how='left')

In [10]:
# --- 8. Feature Engineering for Recommendations ---
# Example features
events_merged['is_view'] = (events_merged['event'] == 'view').astype(int)
events_merged['is_addtocart'] = (events_merged['event'] == 'addtocart').astype(int)
events_merged['is_transaction'] = (events_merged['event'] == 'transaction').astype(int)

# Aggregate per user-item pair
user_item_features = events_merged.groupby(['visitorid', 'itemid']).agg({
    'is_view': 'sum',
    'is_addtocart': 'sum',
    'is_transaction': 'sum',
    'categoryid': 'first',
    'available': 'first'
}).reset_index()

In [11]:
# --- 9. Feature Engineering for Anomaly Detection ---
user_features = events_merged.groupby('visitorid').agg({
    'itemid': 'nunique',
    'is_view': 'sum',
    'is_addtocart': 'sum',
    'is_transaction': 'sum',
    'categoryid': 'nunique'
}).reset_index()

user_features.rename(columns={
    'itemid': 'unique_items_viewed',
    'is_view': 'total_views',
    'is_addtocart': 'total_addtocarts',
    'is_transaction': 'total_transactions',
    'categoryid': 'unique_categories'
}, inplace=True)

# Add ratio features
user_features['view_to_cart_ratio'] = user_features['total_addtocarts'] / user_features['total_views'].replace(0, 1)
user_features['cart_to_purchase_ratio'] = user_features['total_transactions'] / user_features['total_addtocarts'].replace(0, 1)


In [12]:
# Keep only last property for each item (example)
item_features = item_props.sort_values("timestamp").drop_duplicates(["itemid", "property"], keep="last")

# Pivot so each property becomes a column
item_features = item_features.pivot(index="itemid", columns="property", values="value").reset_index()

In [ ]:
# --- 10. Save Processed Data ---
events_merged.to_csv("../content/drive/MyDrive/data/TMP/processed/events_merged.csv", index=False)
user_item_features.to_csv("../content/drive/MyDrive/data/TMP/processed/user_item_features.csv", index=False)
user_features.to_csv("../content/drive/MyDrive/data/TMP/processed/user_features.csv", index=False)
item_features.to_csv("../content/drive/MyDrive/data/TMP/processed/item_features.csv", index=False)

print("Data preparation complete. Files saved to ../TMP/processed/")

Data preparation complete. Files saved to ../TMP/processed/
